In [1]:
import requests
from bs4 import BeautifulSoup
import re

def extract_h5_tags(url, muted):
    results = []
    try:
        # Fetch the webpage content
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all h5 tags
        h5_tags = [tag for tag in soup.find_all('h5') if re.search(r'\d', tag.text)]
        
        if muted :
            #We suppose 5 first are winning numbers and two last are stars
            for i, num in enumerate(h5_tags[0:5], 1):
                results.append(num.text.strip())
            for i, num in enumerate(h5_tags[5:7], 1):
                results.append(num.text.strip())
            
        else :# Print the results
            print(f"Found {len(h5_tags)} h5 tags with numbers only on {url}:")
            for i, num in enumerate(h5_tags[0:5], 1):
                print(f"Winning number {i} : {num.text.strip()}")
                results.append(num.text.strip())
            for i, num in enumerate(h5_tags[5:7], 1):
                print(f"Winning star number {i} : {num.text.strip()}")
                results.append(num.text.strip())
            
        return results
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return results

# Example usage
url_to_scrape = "https://www.fdj.fr/jeux-de-tirage/euromillions-my-million/resultats/mardi-25-mars-2025"  # Replace with your target URL
extract_h5_tags(url_to_scrape, 1)

['1', '3', '18', '33', '36', '2', '6']

In [8]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import locale
import unicodedata

def get_tuesdays_fridays_french():
    """
    Get all Tuesdays and Fridays between two dates in French format 'jour-JJ-mois-AAAA'.
    
    Args:
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format
    
    Returns:
        list: List of date objects
        list: List of formatted date strings in French
    """
    # Set French locale
    locale.setlocale(locale.LC_TIME, 'fr_FR')
    
    # Get dates and 2 months ago
    start = datetime.today() - relativedelta(months=2)
    end = datetime.today() if datetime.today().now().hour > 20 else datetime.today() - relativedelta(days=1)
    
    # Initialize result list
    result = []
    iso_dates = []
    
    # Iterate through each day in the range
    current = start
    while current <= end:
        # Check if current day is Tuesday (1) or Friday (4)
        if current.weekday() in [1, 4]:
            # Format as "jour-JJ-mois-AAAA" (e.g. "Mardi-15-Octobre-2024")
            # Removes accented characters
            nfkd_form = unicodedata.normalize("NFKD",current.strftime("%A-%d-%B-%Y"))
            formatted_date = "".join([c for c in nfkd_form if not unicodedata.combining(c)])
            result.append(formatted_date)
            iso_dates.append(current.strftime("%Y-%m-%d"))
        
        # Move to next day
        current += timedelta(days=1)
    
    # Reset locale to default
    locale.setlocale(locale.LC_TIME, '')
    
    return iso_dates, result

# Example usage
iso_dates, french_dates = get_tuesdays_fridays_french()

In [9]:
%run selfdb.ipynb

def save_to_db(resultats, date, db_path="euromillions.db"):
    try:
        initialize_database(db_path)
        conn = sqlite3.connect(db_path)
        add_calendar_date(conn, date)
        for num in resultats[0:5]:
            add_number(conn, date, num)
        for num in resultats[5:]:
            add_star(conn, date, num)
        conn.close()
        print(f"Saved date : {date}")
        return 0
    except Exception as e :
        print(f"Error in saving : {e}")
        return 1

In [10]:
base_url = "https://www.fdj.fr/jeux-de-tirage/euromillions-my-million/resultats/"
for i, french_date in enumerate(french_dates, 1):
    date_results = extract_h5_tags(base_url+french_date, muted=1)
    save_to_db(date_results,iso_dates[i-1])

Saved date : 2025-02-11
Saved date : 2025-02-14
Saved date : 2025-02-18
Saved date : 2025-02-21
Saved date : 2025-02-25
Saved date : 2025-02-28
Saved date : 2025-03-04
Saved date : 2025-03-07
Saved date : 2025-03-11
Saved date : 2025-03-14
Saved date : 2025-03-18
Saved date : 2025-03-21
Saved date : 2025-03-25
Saved date : 2025-03-28
Saved date : 2025-04-01
Saved date : 2025-04-04
Saved date : 2025-04-08
